In [41]:
import json
data = json.load(open(r"C:\Users\thanh\Downloads\jobs.merged_collections_1.json",'r',encoding='utf-8'))


In [10]:
!pip install datasketch

  Obtaining dependency information for datasketch from https://files.pythonhosted.org/packages/81/71/fb0c28eff49fc0d725782f6dcf4ba2f71c52b6e6e9575179df3802b19d90/datasketch-1.6.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/88.3 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/88.3 kB ? eta -:--:--
   ------------------------------------- -- 81.9/88.3 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 88.3/88.3 kB 1.3 MB/s eta 0:00:00


In [11]:
from datasketch import MinHash, MinHashLSH

set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)
print("Approximate neighbours with Jaccard similarity > 0.5", result)

Approximate neighbours with Jaccard similarity > 0.5 ['m2', 'm3']


In [47]:
texts = []
for x in data:
    text = ''
    for key in list(x.keys()):
        if type(x[key]) == str:
            text += x[key]
        elif type(x[key]) == list:
            text += ' ' + ' '.join(x[key])
    texts.append(text)
texts = [set(i.split()) for i in texts]


In [48]:
# deduplicate using minhash + LSH if jaccard similarity > 0.8
minhashes = {}
lsh = MinHashLSH(threshold=0.8, num_perm=128)
for i, req in enumerate(texts):
    m = MinHash(num_perm=128)
    for d in req:
        m.update(d.encode('utf8'))
    minhashes[i] = m
    lsh.insert(i, m)

In [49]:
# find duplicates
duplicates = []
for i, req in enumerate(texts):
    result = lsh.query(minhashes[i])
    if len(result) > 1:
        duplicates.append(result)
        

In [ ]:
for x in duplicates:
    if len(x) > 1:
        print(x)

In [53]:
data[65]

{'_id': {'$oid': '657882c681ab5b1bd0570b3e'},
 'keyword': 'back-end developer',
 'job_name': 'ANDROID DEVELOPER (LẬP TRÌNH VIÊN)',
 'requirements': [],
 'salary': ['2 times/year', '34000K - 34000K VND một tháng'],
 'offer': ['Chăm Sóc Sức Khỏe, Chế độ bảo hiểm, Thưởng doanh thu, Hỗ trợ Laptop'],
 'location': 'Ho Chi Minh',
 'company': 'Việc Ơi It Client',
 'from': 'indeed'}

In [55]:
# remove duplicates
for i in duplicates:
    for j in i[1:]:
        data[j]['requirements'] = []
        data[j]['from'] = 'duplicate'
        

In [56]:
new_data = [i for i in data if i['from'] != 'duplicate']
len(new_data)

11967

In [36]:
len(data)

14509

In [63]:
# save to json
with open('./data_deduplicated.json','w',encoding='utf-8') as f:
    json.dump(new_data,f,ensure_ascii=False,indent=4)